In [2]:
import pandas as pd
import numpy as np

In [3]:
# Sample Product DataFrame
product_data = pd.DataFrame({
    'ProductID': [101, 102, 103, 104, 105],
    'ProductName': ['ProductA', 'ProductB', 'ProductC', 'ProductD', 'ProductE'],
    'Category': ['Electronics', 'Furniture', 'Electronics', 'Furniture', 'Electronics']
})

# Sample Sales DataFrame
sales_data = pd.DataFrame({
    'SalesID': [1, 2, 3, 4, 5, 6, 7],
    'ProductID': [101, 102, 103, 101, 104, 105, 102],
    'CustomerID': [201, 202, 203, 204, 205, 206, 207],
    'Date': ['2024-10-01', '2024-10-02', '2024-10-03', '2024-10-04', '2024-10-05', '2024-10-06', '2024-10-07'],
    'UnitsSold': [5, 3, 8, 2, 4, 1, 7],
    'Revenue': [100, 60, 240, 40, 160, 50, 140]
})

# Sample Customer DataFrame
customer_data = pd.DataFrame({
    'CustomerID': [201, 202, 203, 204, 205, 206, 207],
    'Region': ['North', 'South', 'East', 'West', 'North', 'South', 'East'],
    'CustomerType': ['Regular', 'Premium', 'Regular', 'Premium', 'Regular', 'Regular', 'Premium']
})

# Sample Discount DataFrame
discount_data = pd.DataFrame({
    'ProductID': [101, 103, 104],
    'Discount': [0.1, 0.15, 0.2],
    'StartDate': ['2024-10-01', '2024-10-03', '2024-10-05'],
    'EndDate': ['2024-10-10', '2024-10-10', '2024-10-10']
})

In [4]:
# 1. Merging Sales and Product DataFrames
merged_sales_product = pd.merge(sales_data, product_data, on='ProductID', how='inner')

In [5]:
# 2. Merging with Customer DataFrame
merged_sales_customer = pd.merge(merged_sales_product, customer_data, on='CustomerID', how='inner')

In [6]:
# 3. Merging with Discount DataFrame to apply discount if applicable
merged_sales_discount = pd.merge(merged_sales_customer, discount_data, on='ProductID', how='left')

In [7]:
# 4. Applying discount logic: Check if the sale date falls within the discount period
merged_sales_discount['Date'] = pd.to_datetime(merged_sales_discount['Date'])
merged_sales_discount['StartDate'] = pd.to_datetime(merged_sales_discount['StartDate'])
merged_sales_discount['EndDate'] = pd.to_datetime(merged_sales_discount['EndDate'])
merged_sales_discount['DiscountApplied'] = np.where(
    (merged_sales_discount['Date'] >= merged_sales_discount['StartDate']) & 
    (merged_sales_discount['Date'] <= merged_sales_discount['EndDate']),
    merged_sales_discount['Discount'],
    0
)

In [8]:
# 5. Calculating total revenue after discount
merged_sales_discount['DiscountedRevenue'] = merged_sales_discount['Revenue'] * (1 - merged_sales_discount['DiscountApplied'])

In [9]:
# 6. Creating new features: Average revenue per unit sold
merged_sales_discount['AvgRevenuePerUnit'] = merged_sales_discount['Revenue'] / merged_sales_discount['UnitsSold']

In [10]:
# 7. Reshaping data: Pivot to see total revenue per product by customer type
pivoted_data = merged_sales_discount.pivot_table(
    index='ProductName', 
    columns='CustomerType', 
    values='DiscountedRevenue', 
    aggfunc='sum', 
    fill_value=0
)

In [11]:
# 8. Creating a summary of total sales by region
sales_by_region = merged_sales_discount.groupby('Region')['DiscountedRevenue'].sum().reset_index()

In [12]:
# Final DataFrame output
print("Merged Sales Data with Discount Applied:\n", merged_sales_discount)
print("\nPivoted Data - Total Revenue per Product by Customer Type:\n", pivoted_data)
print("\nSales Summary by Region:\n", sales_by_region)

Merged Sales Data with Discount Applied:
    SalesID  ProductID  CustomerID       Date  UnitsSold  Revenue ProductName  \
0        1        101         201 2024-10-01          5      100    ProductA   
1        4        101         204 2024-10-04          2       40    ProductA   
2        2        102         202 2024-10-02          3       60    ProductB   
3        7        102         207 2024-10-07          7      140    ProductB   
4        3        103         203 2024-10-03          8      240    ProductC   
5        5        104         205 2024-10-05          4      160    ProductD   
6        6        105         206 2024-10-06          1       50    ProductE   

      Category Region CustomerType  Discount  StartDate    EndDate  \
0  Electronics  North      Regular      0.10 2024-10-01 2024-10-10   
1  Electronics   West      Premium      0.10 2024-10-01 2024-10-10   
2    Furniture  South      Premium       NaN        NaT        NaT   
3    Furniture   East      Premium   